# Train Classifier
The following notebook trains a number of classifiers for hand gesture recognition

Import the necessary dependencies

In [ ]:
import pandas as pd
# ML
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# Accuracy metrics 
from sklearn.metrics import accuracy_score 
# Loading and saving the models
import pickle 


### Load the data
Data on 3 gestures (peace, rock on and thumbs up) was collected, also data classified as other was colledcted. The "other" data was a collection of other random hand gestures excluding the gestures that were to be classified.

In [ ]:
from pathlib import Path
csv_path_peace = Path.cwd().joinpath("data","gesture_database","gestures_peace.csv")
csv_path_rock_on = Path.cwd().joinpath("data","gesture_database","gestures_rock_on.csv")
csv_path_thumb_up = Path.cwd().joinpath("data","gesture_database","gestures_thumb_up.csv")
csv_path_other = Path.cwd().joinpath("data","gesture_database","gestures_other.csv")
df_peace = pd.read_csv(csv_path_peace)
df_rock_on = pd.read_csv(csv_path_rock_on)
df_thumb_up = pd.read_csv(csv_path_thumb_up)
df_other = pd.read_csv(csv_path_other)

Combine the training data into a singe dataframe.


In [ ]:
df_peace.insert(0,"class","peace")
df_rock_on.insert(0,"class","rock_on")
df_thumb_up.insert(0,"class","thumb_up")
df_other.insert(0,"class","other")
frames = [df_peace,df_rock_on,df_thumb_up,df_other]
df = pd.concat(frames)
df["class"].value_counts()

# Create the Classification models

In [ ]:
# correct the column names
df.columns = ['class','x0','y0','z0','x1','y1','z1','x2','y2','z2','x3','y3','z3','x4','y4','z4','x5','y5','z5','x6','y6','z6','x7','y7','z7','x8','y8','z8','x9','y9','z9','x10','y10','z10','x11','y11','z11','x12','y12','z12','x13','y13','z13','x14','y14','z14','x15','y15','z15','x16','y16','z16','x17','y17','z17','x18','y18','z18','x19','y19','z19','x20','y20','z20']
data_columns = df.columns.to_list()[1:] # seaparate just the data columns

In [ ]:
# Split the data columns and the class columns
X = df.loc[:,data_columns].copy(deep=True)    # features - xyz columns
y = df.loc[:,"class"]                          # class

Split the data set to test and train datasets

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

### Train Machine Learning Classification Model

In [ ]:
# Set the piplines for training the models
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

Train the models

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

### Evaluate and Serialize Model 

In [ ]:
results = {} # dictionary of the results
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    results[algo] = round(accuracy_score(y_test, yhat),2)*100
print(results)

In [ ]:
# Save the models
for model in results:
    model_path =  Path.cwd().joinpath("data","models",f"gestures_{model}_{results[model]}.pkl")
    with open(model_path, 'wb') as f:
        pickle.dump(fit_models[model], f)